In [1]:
import pandas as pd
import pickle as pkl
import tensorflow as tf
import time

/home/ubuntu/miniconda3/envs/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def model_fn(features, labels, mode):
    emb_size = 50
    filters=10
    kernel_size=3
    
    with open("data/dictionary.pkl", "rb") as f:
        dictionary = pkl.load(f)
    
    values = list(range(len(dictionary)))
    keys = list(dictionary)
    
    table = tf.contrib.lookup.HashTable(
      tf.contrib.lookup.KeyValueTensorInitializer(keys, values), -1
    )
    
    word_embeddings = tf.get_variable(
        "word_embeddings",
        shape=[len(dictionary), emb_size]
    )
    
    u_inputs = features[0]
    i_inputs = features[1]
    
    u_inputs = table.lookup(u_inputs)
    i_inputs = table.lookup(i_inputs)

    u_inputs = tf.nn.embedding_lookup(word_embeddings, u_inputs)
    i_inputs = tf.nn.embedding_lookup(word_embeddings, i_inputs)
    
    user_conv1 = tf.layers.conv1d(
        u_inputs,
        filters,
        kernel_size,
        use_bias=True,
        activation=tf.nn.tanh,
        name="user_conv")

    item_conv1 = tf.layers.conv1d(
        i_inputs,
        filters,
        kernel_size,
        use_bias=True,
        activation=tf.nn.tanh,
        name="item_conv")

    user_max_pool1 = tf.layers.max_pooling1d(user_conv1, 2, 1)
    item_max_pool1 = tf.layers.max_pooling1d(item_conv1, 2, 1)

    user_flat = tf.layers.flatten(user_max_pool1)
    item_flat = tf.layers.flatten(item_max_pool1)

    user_dense = tf.layers.dense(user_flat, 64, activation=tf.nn.relu)
    item_dense = tf.layers.dense(item_flat, 64, activation=tf.nn.relu)
    
    predictions = tf.reduce_sum( tf.multiply( user_dense, item_dense ), 1, keep_dims=True )
    
    output = {
        "rating": predictions,
        "user_review_embedding": user_flat,
        "item_review_embedding": item_flat
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)


    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.mean_squared_error(labels, predictions)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "mean square error": tf.metrics.mean_squared_error(
            labels=labels, predictions=predictions)
    }
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

# Test Forward

In [3]:
truncate_len = 800
batch_size = 32

In [4]:
def get_truncate_fn(trunc_len):
    def truncate_fn(user, item, rating):
        return user[:trunc_len], item[:trunc_len], rating
    return truncate_fn

In [5]:
def split_fn(user, item, rating):
    user = tf.string_split(user)
    item = tf.string_split(item)
    return user.values, item.values, rating

In [6]:
def parse_fn(record):
    features = {
            "user_review": tf.FixedLenSequenceFeature([], tf.string, allow_missing=True),
            "item_review": tf.FixedLenSequenceFeature([], tf.string, allow_missing=True),
            "rating": tf.FixedLenFeature([1], tf.float32)
        }
    parsed_features = tf.parse_single_example(record, features)
    return parsed_features["user_review"], parsed_features["item_review"], parsed_features["rating"]

In [7]:
def get_dataset_iterator(loc, batch_size, max_len, pad_value):
    dataset = tf.data.TFRecordDataset(loc)
    dataset = dataset.map(parse_fn, num_parallel_calls=batch_size)
    dataset = dataset.map(split_fn, num_parallel_calls=batch_size)
    dataset = dataset.map(get_truncate_fn(max_len), num_parallel_calls=batch_size)
    dataset = dataset.padded_batch(batch_size, padded_shapes=([max_len], [max_len], [None]), padding_values=(pad_value, pad_value, 0.0))
    iterator = dataset.make_one_shot_iterator()
    return iterator

In [8]:
def train_input_fn():
    train_dataset = get_dataset_iterator(
        loc="data/train.tfrecords",
        batch_size=batch_size,
        max_len=truncate_len,
        pad_value="unk")
    nex = train_dataset.get_next()
    return (nex[0], nex[1]), tf.cast(nex[2], tf.int32)

In [9]:
def test_input_fn():
    test_dataset = get_dataset_iterator(
        loc="data/test.tfrecords",
        batch_size=batch_size,
        max_len=truncate_len,
        pad_value="unk"
    )
    nex = test_dataset.get_next()
    return (nex[0], nex[1]), nex[2]

In [10]:
scoring_function = tf.estimator.Estimator(
    model_fn=model_fn,
    model_dir="output/model_" + str(int(time.time())))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'output/model_1523831640', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f79a941e2b0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [11]:
# from tensorflow.python import debug as tf_debug
# hook = tf_debug.TensorBoardDebugHook("localhost:6060")
s = time.time()
scoring_function.train(input_fn=train_input_fn) #, hooks=[hook])
e = time.time()
print("ran {} seconds".format(e - s))

INFO:tensorflow:Calling model_fn.
Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into output/model_1523831640/model.ckpt.
INFO:tensorflow:loss = 15.340349, step = 1
INFO:tensorflow:global_step/sec: 33.9628
INFO:tensorflow:loss = 1.4795471, step = 101 (2.946 sec)
INFO:tensorflow:global_step/sec: 37.0581
INFO:tensorflow:loss = 1.4250457, step = 201 (2.698 sec)
INFO:tensorflow:global_step/sec: 34.7622
INFO:tensorflow:loss = 0.78281665, step = 301 (2.877 sec)
INFO:tensorflow:global_step/sec: 35.3997
INFO:tensorflow:loss = 0.61032724, step = 401 (2.826 sec)
INFO:tensorflow:global_step/sec: 39.2073
INFO:tensorflow:loss = 1.0034945, step = 501 (2

In [12]:
scoring_function.evaluate(input_fn=test_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-15-22:34:28
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from output/model_1523831640/model.ckpt-824
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-15-22:34:45
INFO:tensorflow:Saving dict for global step 824: global_step = 824, loss = 1.0663562, mean square error = 1.0665209


{'global_step': 824, 'loss': 1.0663562, 'mean square error': 1.0665209}

In [13]:
action_movie_lover = """i really love action movies they are my favorite kind of movie because i love to watch the good guys
win. some of my favorite actors are jet li and jackie chan because they were the last great actors who
actually knew how to fight. modern action movie actors are just pretty faces and the editors swap camera
angles when supposed hits make contact"""
action_movie_hater = """I really hate action movies. jackie chan and jet li are the worst. their old fashioned
special effects, if you can even call them that, are out dated and boring. why are people even still paying
for them to make movies"""
item = """this movie is great because of the way the strong, clever, hero saves the day at the end. as always
jackie chans action directing and stunts are amazing, i'm so glad he doesn't use a stunt double liek
some other actors that don't need to be mentioned"""


In [14]:
def get_predict_input_fn(user, item, trun_len):
    def predict_input_fn():
        original_user_review = [bytes(v, "utf8") for v in user.split()]
        original_item_review = [bytes(v, "utf8") for v in item.split()]
        r_user = tf.constant([original_user_review + [b"unk"] * (trun_len - len(original_user_review))])
        r_item = tf.constant([original_item_review + [b"unk"] * (trun_len - len(original_item_review))])
        return (r_user, r_item), None
    return predict_input_fn

In [15]:
prediction = scoring_function.predict(input_fn=get_predict_input_fn(action_movie_lover, item, truncate_len))
print(next(prediction))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from output/model_1523831640/model.ckpt-824
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[3.1865366]


In [16]:
prediction = scoring_function.predict(input_fn=get_predict_input_fn(action_movie_hater, item, truncate_len))
print(next(prediction))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from output/model_1523831640/model.ckpt-824
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[2.827323]
